In [33]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import tensorflow as tf
from transformers import pipeline

In [34]:
df = pd.read_csv("data_sets/tweets_douplicates_removed.csv.gz")

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47690 entries, 0 to 47689
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   0                 47690 non-null  int64 
 1   text              47690 non-null  object
 2   screen_name       47690 non-null  object
 3   user_description  39352 non-null  object
 4   favourite_count   47690 non-null  int64 
 5   retweet_count     47690 non-null  int64 
 6   created_at        47690 non-null  object
 7   replying_to       1172 non-null   object
 8   media             47690 non-null  bool  
 9   hashtags          47690 non-null  object
 10  urls              47690 non-null  object
 11  user_mentions     47690 non-null  object
 12  is_quote          47690 non-null  bool  
 13  is_retweet        47690 non-null  bool  
dtypes: bool(3), int64(3), object(8)
memory usage: 4.1+ MB


In [36]:
df.head()

,0,text,screen_name,user_description,favourite_count,retweet_count,created_at,replying_to,media,hashtags,urls,user_mentions,is_quote,is_retweet
0,0,"RT @USNationalGuard: Today, approximately 5,20...",echristensen113,"Native West Texan, avid gardener, skilled chef.",0,92,2022-10-03 20:19:51+00:00,NaN,False,[],[],"[{'screen_name': 'USNationalGuard', 'name': 'N...",False,True
1,1,"RT @glamelegance: Is it just me, or does anyon...",BlanketFtBliss,NaN,0,5452,2022-10-03 20:19:50+00:00,NaN,False,[],[],"[{'screen_name': 'glamelegance', 'name': 'Jule...",False,True
2,2,"RT @HomeDepotFound: Over the weekend, one of o...",EBadger76,Asset Protection Supervisor at Redlands 5087! ...,0,29,2022-10-03 20:19:46+00:00,NaN,False,[],[],"[{'screen_name': 'HomeDepotFound', 'name': 'Th...",False,True
3,3,RT @TeamPelosi: ALL House Democrats said YES t...,kenneyy88,NaN,0,6449,2022-10-03 20:19:45+00:00,NaN,False,[],[],"[{'screen_name': 'TeamPelosi', 'name': 'Nancy ...",False,True
4,4,“#Florida's death toll from #HurricaneIan tops...,AmPowerBlog,Sports Twitter is the best Twitter. 🏈🏇🎾🛹⚾🏌️😎🚴🏐...,0,0,2022-10-03 20:19:43+00:00,NaN,False,"[{'text': 'Florida', 'indices': [1, 9]}, {'tex...","[{'url': 'https://t.co/RqcyAHAxtk', 'expanded_...",[],False,False


In [37]:
is_retweet = []

for tweet in df['text']:
    if tweet.startswith("RT"):
        is_retweet.append(True)
    else:
        is_retweet.append(False)

In [38]:
df['is_retweet'] = is_retweet
df.head()

,0,text,screen_name,user_description,favourite_count,retweet_count,created_at,replying_to,media,hashtags,urls,user_mentions,is_quote,is_retweet
0,0,"RT @USNationalGuard: Today, approximately 5,20...",echristensen113,"Native West Texan, avid gardener, skilled chef.",0,92,2022-10-03 20:19:51+00:00,NaN,False,[],[],"[{'screen_name': 'USNationalGuard', 'name': 'N...",False,True
1,1,"RT @glamelegance: Is it just me, or does anyon...",BlanketFtBliss,NaN,0,5452,2022-10-03 20:19:50+00:00,NaN,False,[],[],"[{'screen_name': 'glamelegance', 'name': 'Jule...",False,True
2,2,"RT @HomeDepotFound: Over the weekend, one of o...",EBadger76,Asset Protection Supervisor at Redlands 5087! ...,0,29,2022-10-03 20:19:46+00:00,NaN,False,[],[],"[{'screen_name': 'HomeDepotFound', 'name': 'Th...",False,True
3,3,RT @TeamPelosi: ALL House Democrats said YES t...,kenneyy88,NaN,0,6449,2022-10-03 20:19:45+00:00,NaN,False,[],[],"[{'screen_name': 'TeamPelosi', 'name': 'Nancy ...",False,True
4,4,“#Florida's death toll from #HurricaneIan tops...,AmPowerBlog,Sports Twitter is the best Twitter. 🏈🏇🎾🛹⚾🏌️😎🚴🏐...,0,0,2022-10-03 20:19:43+00:00,NaN,False,"[{'text': 'Florida', 'indices': [1, 9]}, {'tex...","[{'url': 'https://t.co/RqcyAHAxtk', 'expanded_...",[],False,False


In [39]:
df = df.loc[df['is_retweet'] == False]
df.shape

(7652, 14)

In [40]:
df = df.drop(columns = '0')

# Vader

In [41]:
analyzer = SentimentIntensityAnalyzer()

vader = [analyzer.polarity_scores(x) for x in df['text']]

In [42]:
df['vader'] = vader
df.head()

,text,screen_name,user_description,favourite_count,retweet_count,created_at,replying_to,media,hashtags,urls,user_mentions,is_quote,is_retweet,vader
4,“#Florida's death toll from #HurricaneIan tops...,AmPowerBlog,Sports Twitter is the best Twitter. 🏈🏇🎾🛹⚾🏌️😎🚴🏐...,0,0,2022-10-03 20:19:43+00:00,NaN,False,"[{'text': 'Florida', 'indices': [1, 9]}, {'tex...","[{'url': 'https://t.co/RqcyAHAxtk', 'expanded_...",[],False,False,"{'neg': 0.176, 'neu': 0.676, 'pos': 0.149, 'co..."
11,Republicans. can’t. be. counted. on. to. do. ...,nivnos33,#RESISTER #Woke #Democrat #NeverGOP #VotingRig...,0,0,2022-10-03 20:19:22+00:00,NaN,False,"[{'text': 'VoteOutEveryRepublican', 'indices':...","[{'url': 'https://t.co/Me3qmrzTsX', 'expanded_...",[],True,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
16,Leadership you can Trust. 🦟 Make sure to like ...,TrishTheCommish,"#Commissioner, #Mom, #PublicServant, #Mosquito...",2,0,2022-10-03 20:19:09+00:00,NaN,True,"[{'text': 'leadbyexample', 'indices': [180, 19...",[],[],False,False,"{'neg': 0.0, 'neu': 0.719, 'pos': 0.281, 'comp..."
23,"Hello Everyone,\n1/3) Many Floridians face flo...",Find_and_Bind1,"Amateur journalist, photographer, #bondage ent...",0,0,2022-10-03 20:18:56+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [112, 125...","[{'url': 'https://t.co/lgO1y1sFsK', 'expanded_...",[],False,False,"{'neg': 0.139, 'neu': 0.779, 'pos': 0.082, 'co..."
28,"Lord, please be a refuge for those in need. Gi...",shellsfaith,My name is Shelly and this is where I will be ...,1,0,2022-10-03 20:18:45+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [195, 208...","[{'url': 'https://t.co/M4c6nH2x1U', 'expanded_...",[],True,False,"{'neg': 0.093, 'neu': 0.692, 'pos': 0.215, 'co..."


# TextBlob

In [43]:
text_blob = []
for x in df['text']:
    blob = TextBlob(x)
    text_blob.append(blob.sentiment.polarity)
    
df['text_blob'] = text_blob

In [44]:
df.head()

,text,screen_name,user_description,favourite_count,retweet_count,created_at,replying_to,media,hashtags,urls,user_mentions,is_quote,is_retweet,vader,text_blob
4,“#Florida's death toll from #HurricaneIan tops...,AmPowerBlog,Sports Twitter is the best Twitter. 🏈🏇🎾🛹⚾🏌️😎🚴🏐...,0,0,2022-10-03 20:19:43+00:00,NaN,False,"[{'text': 'Florida', 'indices': [1, 9]}, {'tex...","[{'url': 'https://t.co/RqcyAHAxtk', 'expanded_...",[],False,False,"{'neg': 0.176, 'neu': 0.676, 'pos': 0.149, 'co...",0.000000
11,Republicans. can’t. be. counted. on. to. do. ...,nivnos33,#RESISTER #Woke #Democrat #NeverGOP #VotingRig...,0,0,2022-10-03 20:19:22+00:00,NaN,False,"[{'text': 'VoteOutEveryRepublican', 'indices':...","[{'url': 'https://t.co/Me3qmrzTsX', 'expanded_...",[],True,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.285714
16,Leadership you can Trust. 🦟 Make sure to like ...,TrishTheCommish,"#Commissioner, #Mom, #PublicServant, #Mosquito...",2,0,2022-10-03 20:19:09+00:00,NaN,True,"[{'text': 'leadbyexample', 'indices': [180, 19...",[],[],False,False,"{'neg': 0.0, 'neu': 0.719, 'pos': 0.281, 'comp...",0.625000
23,"Hello Everyone,\n1/3) Many Floridians face flo...",Find_and_Bind1,"Amateur journalist, photographer, #bondage ent...",0,0,2022-10-03 20:18:56+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [112, 125...","[{'url': 'https://t.co/lgO1y1sFsK', 'expanded_...",[],False,False,"{'neg': 0.139, 'neu': 0.779, 'pos': 0.082, 'co...",0.500000
28,"Lord, please be a refuge for those in need. Gi...",shellsfaith,My name is Shelly and this is where I will be ...,1,0,2022-10-03 20:18:45+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [195, 208...","[{'url': 'https://t.co/M4c6nH2x1U', 'expanded_...",[],True,False,"{'neg': 0.093, 'neu': 0.692, 'pos': 0.215, 'co...",-0.200000


# BERT

In [45]:
cardiffnlp = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

2022-11-02 14:13:51.354061: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-02 14:13:51.354542: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
bert = []
for x in df['text']:
    label = cardiffnlp(x)
    bert.append(label)
    
df['bert'] = bert

In [47]:
df.head()

,text,screen_name,user_description,favourite_count,retweet_count,created_at,replying_to,media,hashtags,urls,user_mentions,is_quote,is_retweet,vader,text_blob,bert
4,“#Florida's death toll from #HurricaneIan tops...,AmPowerBlog,Sports Twitter is the best Twitter. 🏈🏇🎾🛹⚾🏌️😎🚴🏐...,0,0,2022-10-03 20:19:43+00:00,NaN,False,"[{'text': 'Florida', 'indices': [1, 9]}, {'tex...","[{'url': 'https://t.co/RqcyAHAxtk', 'expanded_...",[],False,False,"{'neg': 0.176, 'neu': 0.676, 'pos': 0.149, 'co...",0.000000,"[{'label': 'Positive', 'score': 0.352827787399..."
11,Republicans. can’t. be. counted. on. to. do. ...,nivnos33,#RESISTER #Woke #Democrat #NeverGOP #VotingRig...,0,0,2022-10-03 20:19:22+00:00,NaN,False,"[{'text': 'VoteOutEveryRepublican', 'indices':...","[{'url': 'https://t.co/Me3qmrzTsX', 'expanded_...",[],True,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.285714,"[{'label': 'Neutral', 'score': 0.3550549745559..."
16,Leadership you can Trust. 🦟 Make sure to like ...,TrishTheCommish,"#Commissioner, #Mom, #PublicServant, #Mosquito...",2,0,2022-10-03 20:19:09+00:00,NaN,True,"[{'text': 'leadbyexample', 'indices': [180, 19...",[],[],False,False,"{'neg': 0.0, 'neu': 0.719, 'pos': 0.281, 'comp...",0.625000,"[{'label': 'Neutral', 'score': 0.4122076034545..."
23,"Hello Everyone,\n1/3) Many Floridians face flo...",Find_and_Bind1,"Amateur journalist, photographer, #bondage ent...",0,0,2022-10-03 20:18:56+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [112, 125...","[{'url': 'https://t.co/lgO1y1sFsK', 'expanded_...",[],False,False,"{'neg': 0.139, 'neu': 0.779, 'pos': 0.082, 'co...",0.500000,"[{'label': 'Positive', 'score': 0.351958453655..."
28,"Lord, please be a refuge for those in need. Gi...",shellsfaith,My name is Shelly and this is where I will be ...,1,0,2022-10-03 20:18:45+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [195, 208...","[{'url': 'https://t.co/M4c6nH2x1U', 'expanded_...",[],True,False,"{'neg': 0.093, 'neu': 0.692, 'pos': 0.215, 'co...",-0.200000,"[{'label': 'Neutral', 'score': 0.3944049477577..."


In [19]:
df['bert']

4        [{'label': 'Neutral', 'score': 0.3489492237567...
11       [{'label': 'Neutral', 'score': 0.3452346324920...
16       [{'label': 'Negative', 'score': 0.427650004625...
23       [{'label': 'Neutral', 'score': 0.3482158184051...
28       [{'label': 'Negative', 'score': 0.387524873018...
                               ...                        
47595    [{'label': 'Negative', 'score': 0.374450981616...
47613    [{'label': 'Negative', 'score': 0.368252307176...
47615    [{'label': 'Negative', 'score': 0.368372738361...
47629    [{'label': 'Neutral', 'score': 0.3751612305641...
47631    [{'label': 'Negative', 'score': 0.388470619916...
Name: bert, Length: 7652, dtype: object

# Voting

In [48]:
df['vader_compound']  = df['vader'].apply(lambda score_dict: score_dict['compound'])

df.head()


,text,screen_name,user_description,favourite_count,retweet_count,created_at,replying_to,media,hashtags,urls,user_mentions,is_quote,is_retweet,vader,text_blob,bert,vader_compound
4,“#Florida's death toll from #HurricaneIan tops...,AmPowerBlog,Sports Twitter is the best Twitter. 🏈🏇🎾🛹⚾🏌️😎🚴🏐...,0,0,2022-10-03 20:19:43+00:00,NaN,False,"[{'text': 'Florida', 'indices': [1, 9]}, {'tex...","[{'url': 'https://t.co/RqcyAHAxtk', 'expanded_...",[],False,False,"{'neg': 0.176, 'neu': 0.676, 'pos': 0.149, 'co...",0.000000,"[{'label': 'Positive', 'score': 0.352827787399...",-0.1531
11,Republicans. can’t. be. counted. on. to. do. ...,nivnos33,#RESISTER #Woke #Democrat #NeverGOP #VotingRig...,0,0,2022-10-03 20:19:22+00:00,NaN,False,"[{'text': 'VoteOutEveryRepublican', 'indices':...","[{'url': 'https://t.co/Me3qmrzTsX', 'expanded_...",[],True,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.285714,"[{'label': 'Neutral', 'score': 0.3550549745559...",0.0000
16,Leadership you can Trust. 🦟 Make sure to like ...,TrishTheCommish,"#Commissioner, #Mom, #PublicServant, #Mosquito...",2,0,2022-10-03 20:19:09+00:00,NaN,True,"[{'text': 'leadbyexample', 'indices': [180, 19...",[],[],False,False,"{'neg': 0.0, 'neu': 0.719, 'pos': 0.281, 'comp...",0.625000,"[{'label': 'Neutral', 'score': 0.4122076034545...",0.9134
23,"Hello Everyone,\n1/3) Many Floridians face flo...",Find_and_Bind1,"Amateur journalist, photographer, #bondage ent...",0,0,2022-10-03 20:18:56+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [112, 125...","[{'url': 'https://t.co/lgO1y1sFsK', 'expanded_...",[],False,False,"{'neg': 0.139, 'neu': 0.779, 'pos': 0.082, 'co...",0.500000,"[{'label': 'Positive', 'score': 0.351958453655...",-0.3182
28,"Lord, please be a refuge for those in need. Gi...",shellsfaith,My name is Shelly and this is where I will be ...,1,0,2022-10-03 20:18:45+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [195, 208...","[{'url': 'https://t.co/M4c6nH2x1U', 'expanded_...",[],True,False,"{'neg': 0.093, 'neu': 0.692, 'pos': 0.215, 'co...",-0.200000,"[{'label': 'Neutral', 'score': 0.3944049477577...",0.7579


In [49]:
df = df.drop(columns='vader')

In [50]:
bert_label = []

for x in df['bert']:
    bert_label.append(x[0]['label'])

In [51]:
df['bert_label'] = bert_label
df.head()

,text,screen_name,user_description,favourite_count,retweet_count,created_at,replying_to,media,hashtags,urls,user_mentions,is_quote,is_retweet,text_blob,bert,vader_compound,bert_label
4,“#Florida's death toll from #HurricaneIan tops...,AmPowerBlog,Sports Twitter is the best Twitter. 🏈🏇🎾🛹⚾🏌️😎🚴🏐...,0,0,2022-10-03 20:19:43+00:00,NaN,False,"[{'text': 'Florida', 'indices': [1, 9]}, {'tex...","[{'url': 'https://t.co/RqcyAHAxtk', 'expanded_...",[],False,False,0.000000,"[{'label': 'Positive', 'score': 0.352827787399...",-0.1531,Positive
11,Republicans. can’t. be. counted. on. to. do. ...,nivnos33,#RESISTER #Woke #Democrat #NeverGOP #VotingRig...,0,0,2022-10-03 20:19:22+00:00,NaN,False,"[{'text': 'VoteOutEveryRepublican', 'indices':...","[{'url': 'https://t.co/Me3qmrzTsX', 'expanded_...",[],True,False,0.285714,"[{'label': 'Neutral', 'score': 0.3550549745559...",0.0000,Neutral
16,Leadership you can Trust. 🦟 Make sure to like ...,TrishTheCommish,"#Commissioner, #Mom, #PublicServant, #Mosquito...",2,0,2022-10-03 20:19:09+00:00,NaN,True,"[{'text': 'leadbyexample', 'indices': [180, 19...",[],[],False,False,0.625000,"[{'label': 'Neutral', 'score': 0.4122076034545...",0.9134,Neutral
23,"Hello Everyone,\n1/3) Many Floridians face flo...",Find_and_Bind1,"Amateur journalist, photographer, #bondage ent...",0,0,2022-10-03 20:18:56+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [112, 125...","[{'url': 'https://t.co/lgO1y1sFsK', 'expanded_...",[],False,False,0.500000,"[{'label': 'Positive', 'score': 0.351958453655...",-0.3182,Positive
28,"Lord, please be a refuge for those in need. Gi...",shellsfaith,My name is Shelly and this is where I will be ...,1,0,2022-10-03 20:18:45+00:00,NaN,False,"[{'text': 'HurricaneIan', 'indices': [195, 208...","[{'url': 'https://t.co/M4c6nH2x1U', 'expanded_...",[],True,False,-0.200000,"[{'label': 'Neutral', 'score': 0.3944049477577...",0.7579,Neutral


In [52]:
df.to_csv("ready_for_analysis.csv")